# Mapzen python code

You will need to
- install Enthought or Anaconda, or another reader
- install requests and json library

## Always execute these next two cells of code!

This is a MapZen key - you will need to use your own. Please note that you should NEVER share your key to GitHub or elsewhere.

In [233]:
key = 'mapzen-yourkey'
#endpoint = "https://valhalla.mapzen.com/matrix"
#endpoint = "https://matrix.mapzen.com/many_to_many"
 

In [137]:
import requests
import json
import pandas

# 1. Example query:

In [78]:
path=json.dumps({"locations":[{"lat":40.744014,"lon":-73.990508},{"lat":40.739735,"lon":-73.979713},{"lat":40.752522,"lon":-73.985015},{"lat":40.750117,"lon":-73.983704},{"lat":40.750552,"lon":-73.993519}], "costing":"auto"})

In [79]:
#params = {"json": json.dumps(route), "api_key": key}
params = {"json":path,"api_key":key}
req = requests.get(endpoint, params=params)

In [80]:
print req.text

{"locations":[[{"lon":-73.990509,"lat":40.744015},{"lon":-73.979713,"lat":40.739735},{"lon":-73.985016,"lat":40.752522},{"lon":-73.983704,"lat":40.750118},{"lon":-73.993523,"lat":40.750553}]],"units":"km","many_to_many":[[{"distance":0.000,"time":0,"to_index":0,"from_index":0},{"distance":1.346,"time":335,"to_index":1,"from_index":0},{"distance":1.944,"time":341,"to_index":2,"from_index":0},{"distance":1.730,"time":340,"to_index":3,"from_index":0},{"distance":1.202,"time":210,"to_index":4,"from_index":0}],[{"distance":2.047,"time":362,"to_index":0,"from_index":1},{"distance":0.000,"time":0,"to_index":1,"from_index":1},{"distance":2.270,"time":371,"to_index":2,"from_index":1},{"distance":1.903,"time":287,"to_index":3,"from_index":1},{"distance":2.303,"time":390,"to_index":4,"from_index":1}],[{"distance":1.841,"time":348,"to_index":0,"from_index":2},{"distance":2.738,"time":346,"to_index":1,"from_index":2},{"distance":0.000,"time":0,"to_index":2,"from_index":2},{"distance":0.761,"time":2

# 2. Working with a small file (fewer than 50 locations)
## Specify the filename and put it in the "data" folder

Pay attention to the format of the file...

In [213]:
filename = "Example1.csv"

In [214]:
dataIn = pandas.read_csv("data/"+filename)
dataIn.head()

,Node ID,Lat,Lon,Name of delivery point,From\To
0,1,51.523100,-0.260900,Depot,V0
1,2,51.521164,-0.132339,2 ridgmouth garderns,V1
2,3,51.521349,-0.134274,22-24 Torrington place (2nd floor),V2
3,4,51.518203,-0.128535,"21 bloomsbury street london, WC1B 3HF (3rd floor)",V3
4,5,51.518501,-0.124225,"14 bury place, london WC1A 2JL",V4


## Create dict for locations

In [215]:
locations = []
for i in range(len(dataIn.index)):
    locations.append({"lon":dataIn.Lon.iloc[i], "lat":dataIn.Lat.iloc[i]})

test1 = locations

## Specify the travel mode
- auto (car)
- pedestrian
- bicyle

In [179]:
costing = "auto"

## Create a new dict with this and other prefs and feed it into a new json object

In [200]:
dictPath = {"locations":locations, 
            "costing":costing}
#print dictPath

In [181]:
path = json.dumps(dictPath)

## Send a request to MapZen

In [182]:
params = {"json":path,"api_key":key}
req = requests.get(endpoint, params=params)

In [198]:
#print req.url

In [131]:
jsonback = req.json()
locationsOut = jsonback.get("locations")
rawMatrix = jsonback.get("many_to_many")

## Specify Matrix

Create a pandas dataframe, which is easy to save to csv. Here we use "distance", but we could also use "time".

Here the **origin** is the **column**, and the **destination** is the **row**. The matrix is not necessarily symmetric.

In [134]:
cols = list(range(1, 51))
mo = pandas.DataFrame(columns=cols, index = cols)

In [135]:
for row in rawMatrix:
    for edge in row:
        D = edge['to_index']
        O = edge['from_index']
        distance = edge['distance']
        mo.iloc[D,O]= distance
mo.head()

,1,2,3,4,5,6,7,8,9,10,...,41,42,43,44,45,46,47,48,49,50
1,0,10.527,10.326,10.924,11.782,10.912,10.252,10.162,10.065,10.238,...,10.72,10.731,10.369,10.293,10.215,10.322,10.251,9.869,10.238,11.144
2,10.894,0,0.628,0.721,1.6,1.63,0.345,0.345,0.423,0.331,...,1.438,1.449,1.087,1.011,1.555,1.378,1.591,0.925,0.331,0.941
3,10.855,1.364,0,0.989,1.561,3.305,1.061,0.971,0.874,1.047,...,3.113,3.124,2.762,2.686,2.608,1.131,2.644,2.262,1.047,1.209
4,11.247,0.629,2.19,0,1.126,2.087,0.957,0.957,1.03,0.943,...,1.895,1.906,1.544,1.468,1.39,1.213,1.426,1.382,0.943,0.474
5,11.66,0.823,2.033,0.613,0,0.71,1.151,1.151,1.224,1.137,...,0.902,1.267,1.273,1.662,1.584,1.407,1.62,1.576,1.137,0.917


In [119]:
mo.to_csv("dataOut/"+filename)

# 3. Larger files (more than 50 locations)

In [ ]:
filename = "Example2"

In [223]:
dataIn = pandas.read_csv("data/"+filename+".csv")
dataIn.head()

,Node ID,Delivery Lat,Deliver Long,Name of delivery point,Unnamed: 4
0,1,51.523100,-0.260900,Depot,V0
1,2,51.521164,-0.132339,2 ridgmouth garderns,V1
2,3,51.521349,-0.134274,22-24 Torrington place (2nd floor),V2
3,4,51.518203,-0.128535,"21 bloomsbury street london, WC1B 3HF (3rd floor)",V3
4,5,51.518501,-0.124225,"14 bury place, london WC1A 2JL",V4


## Split this into multiple location lists (a list of lists)

Make sure your column names match up here

In [224]:
breakup = 1 + (len(dataIn)/50)
locations = []
for i in range(0, breakup):
    locations.append([])
    
latcol = "Delivery Lat"
loncol = 'Deliver Long'

n=0
for i in range(len(dataIn.index)):
    n = i/50
    locations[n].append({"lon":dataIn[loncol].iloc[i], "lat":dataIn[latcol].iloc[i]})

## Generate multiple matrices and save each to a csv

In [230]:
costing = "auto"
api = "https://matrix.mapzen.com/sources_to_targets"

for n in range (0,breakup):
    for m in range (0,breakup):
        dictPath = {"sources":locations[n],
                    "targets":locations[m],
                    "costing":costing,
                    "id": filename}
        path = json.dumps(dictPath)
        params = {"json":path,"api_key":key}
        req = requests.get(endpoint, params=params)
        #print req.url
        jsonback = req.json()
        #locationsOut = jsonback.get("locations")
        rawMatrix = jsonback.get("sources_to_targets")
        cols = list(range(1+n*50, 51+n*50))
        rows = list(range(1+m*50, 51+m*50))
        mo = pandas.DataFrame(columns=cols, index = rows)
        
        for row in rawMatrix:
            for edge in row:
                D = edge['to_index']
                O = edge['from_index']
                distance = edge['distance']
                mo.iloc[D,O]= distance
                
        mo.to_csv("dataOut/"+filename+"_"+str(n)+"_"+str(0)+".csv")
        

## JUNK BELOW - ignore it

In [231]:
costing = "auto"
endpoint = "https://matrix.mapzen.com/sources_to_targets"

dictPath = {"sources":locations[1],
            "targets":locations[1],
            "costing":costing}
print dictPath
path = json.dumps(dictPath)
params = {"json":path,"api_key":key,"id": filename}
req = requests.get(endpoint, params=params)
#print req.url

{'sources': [{'lat': 51.523625399999993, 'lon': -0.13362449999999998}, {'lat': 51.524465399999997, 'lon': -0.13269420000000001}, {'lat': 51.524188000000002, 'lon': -0.13243099999999999}, {'lat': 51.518244000000003, 'lon': -0.12096510000000001}, {'lat': 51.518924669999997, 'lon': -0.13180378000000001}, {'lat': 51.519795000000002, 'lon': -0.1313291}, {'lat': 51.518234020000001, 'lon': -0.12843228000000001}, {'lat': 51.523468700000002, 'lon': -0.12731389999999998}], 'targets': [{'lat': 51.523625399999993, 'lon': -0.13362449999999998}, {'lat': 51.524465399999997, 'lon': -0.13269420000000001}, {'lat': 51.524188000000002, 'lon': -0.13243099999999999}, {'lat': 51.518244000000003, 'lon': -0.12096510000000001}, {'lat': 51.518924669999997, 'lon': -0.13180378000000001}, {'lat': 51.519795000000002, 'lon': -0.1313291}, {'lat': 51.518234020000001, 'lon': -0.12843228000000001}, {'lat': 51.523468700000002, 'lon': -0.12731389999999998}], 'costing': 'auto'}


In [232]:
costing = "auto"
endpoint = "https://matrix.mapzen.com/many_to_many"

loc = locations[1]

dictPath = {"locations":loc,
            "costing":costing}
print dictPath
path = json.dumps(dictPath)
params = {"json":path,"api_key":key,"id": filename}
req = requests.get(endpoint, params=params)
#print req.url

{'locations': [{'lat': 51.523625399999993, 'lon': -0.13362449999999998}, {'lat': 51.524465399999997, 'lon': -0.13269420000000001}, {'lat': 51.524188000000002, 'lon': -0.13243099999999999}, {'lat': 51.518244000000003, 'lon': -0.12096510000000001}, {'lat': 51.518924669999997, 'lon': -0.13180378000000001}, {'lat': 51.519795000000002, 'lon': -0.1313291}, {'lat': 51.518234020000001, 'lon': -0.12843228000000001}, {'lat': 51.523468700000002, 'lon': -0.12731389999999998}], 'costing': 'auto'}
